In [1]:
!pip install transformers


In [2]:
!pip install --upgrade accelerate

  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/6c/35/b3851a3c2d3ead15099defccb50b59c1165eb24dfde298abe4091ffb6cca/accelerate-0.29.1-py3-none-any.whl.metadata
  Using cached accelerate-0.29.1-py3-none-any.whl.metadata (18 kB)
Using cached accelerate-0.29.1-py3-none-any.whl (297 kB)


In [3]:
import accelerate


In [4]:
!pip uninstall accelerate transformers -y
!pip install transformers torch torchtext torchvision

Found existing installation: accelerate 0.29.1
Uninstalling accelerate-0.29.1:
  Successfully uninstalled accelerate-0.29.1
Found existing installation: transformers 4.39.3
Uninstalling transformers-4.39.3:
  Successfully uninstalled transformers-4.39.3
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/15/fc/7b6dd7e1adc0a6407b845ed4be1999e98b6917d0694e57316d140cc85484/transformers-4.39.3-py3-none-any.whl.metadata
  Using cached transformers-4.39.3-py3-none-any.whl.metadata (134 kB)
Using cached transformers-4.39.3-py3-none-any.whl (8.8 MB)


In [5]:
%reload_ext autoreload
%autoreload 2

In [6]:
import torch
import torchtext
import re
import nltk
#from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer



In [7]:
import pandas as pd
df=pd.read_json('gold_practice_test.json', orient='records')


In [8]:
df=df.T
df.head(5)

,Type,Section_id,Primary_id,Statement,Label,Intervention,Causal_type,Secondary_id
14e55d50-c012-4f32-afc5-6f5f7a030667,Single,Adverse Events,NCT00328783,the primary trial documents a comprehensive li...,Contradiction,Contradiction,"[Altering, fef5aedb-21c0-4130-b8a3-cb1df5017c7a]",NaN
ea57b54d-1912-4d9f-bcf7-7f423aed195b,Comparison,Intervention,NCT01572727,"in the primary clinical trial, patients receiv...",Entailment,Paraphrase,"[Preserving, 3ba42f68-a643-4d10-afd0-cbb358d05...",NCT00328783
6c337426-cff6-4f2c-acab-e7bf15f09cc2,Comparison,Adverse Events,NCT01852032,the primary trial and the secondary trial repo...,Contradiction,Text_appended,"[Preserving, c7a953b0-ac96-4e0a-9c1d-98cf8af28...",NCT01830933
141798cc-9851-4bdf-b8a9-ce688db34e72,Single,Eligibility,NCT00438100,ongoing flucytosine treatments are not permitt...,Entailment,Text_appended,"[Preserving, f10f3b84-e039-4a02-a98b-86d149d9a...",NaN
fdbb037a-8084-4267-9143-f989d26726ab,Single,Results,NCT00217399,all the primary trial patients had a minimum o...,Contradiction,Numerical_contradiction,"[Preserving, 985bc5a0-bc21-447f-92f1-150e23ede...",NaN


In [9]:
df.isnull().sum()

Type               0
Section_id         0
Primary_id         0
Statement          0
Label              0
Intervention     200
Causal_type      200
Secondary_id    1526
dtype: int64

In [10]:
df.shape

(2142, 8)

In [11]:
original_dataset = df[df['Intervention'].isnull()]
altered_dataset = df[df['Intervention'].notnull()]

In [12]:
original_dataset.shape


(200, 8)

In [13]:
altered_dataset.shape

(1942, 8)

In [14]:
altered_dataset.head(5)

,Type,Section_id,Primary_id,Statement,Label,Intervention,Causal_type,Secondary_id
14e55d50-c012-4f32-afc5-6f5f7a030667,Single,Adverse Events,NCT00328783,the primary trial documents a comprehensive li...,Contradiction,Contradiction,"[Altering, fef5aedb-21c0-4130-b8a3-cb1df5017c7a]",NaN
ea57b54d-1912-4d9f-bcf7-7f423aed195b,Comparison,Intervention,NCT01572727,"in the primary clinical trial, patients receiv...",Entailment,Paraphrase,"[Preserving, 3ba42f68-a643-4d10-afd0-cbb358d05...",NCT00328783
6c337426-cff6-4f2c-acab-e7bf15f09cc2,Comparison,Adverse Events,NCT01852032,the primary trial and the secondary trial repo...,Contradiction,Text_appended,"[Preserving, c7a953b0-ac96-4e0a-9c1d-98cf8af28...",NCT01830933
141798cc-9851-4bdf-b8a9-ce688db34e72,Single,Eligibility,NCT00438100,ongoing flucytosine treatments are not permitt...,Entailment,Text_appended,"[Preserving, f10f3b84-e039-4a02-a98b-86d149d9a...",NaN
fdbb037a-8084-4267-9143-f989d26726ab,Single,Results,NCT00217399,all the primary trial patients had a minimum o...,Contradiction,Numerical_contradiction,"[Preserving, 985bc5a0-bc21-447f-92f1-150e23ede...",NaN


In [15]:
altered_dataset['Intervention'].unique()

array(['Contradiction', 'Paraphrase', 'Text_appended',
       'Numerical_contradiction', 'Numerical_paraphrase'], dtype=object)

In [16]:
dataset_f = altered_dataset[altered_dataset['Intervention']. isin(['Numerical_contradiction','Contradiction'])]
dataset_c = altered_dataset[altered_dataset['Intervention']. isin(['Numerical_paraphrase','Paraphrase','Text_appended'])]

In [17]:
dataset_f.head(5)

,Type,Section_id,Primary_id,Statement,Label,Intervention,Causal_type,Secondary_id
14e55d50-c012-4f32-afc5-6f5f7a030667,Single,Adverse Events,NCT00328783,the primary trial documents a comprehensive li...,Contradiction,Contradiction,"[Altering, fef5aedb-21c0-4130-b8a3-cb1df5017c7a]",NaN
fdbb037a-8084-4267-9143-f989d26726ab,Single,Results,NCT00217399,all the primary trial patients had a minimum o...,Contradiction,Numerical_contradiction,"[Preserving, 985bc5a0-bc21-447f-92f1-150e23ede...",NaN
64ab5bcd-229a-4aac-8d01-609e514400d7,Single,Intervention,NCT00374322,the patient cohorts in the primary clinical tr...,Contradiction,Contradiction,"[Altering, c40fdf91-9f81-454a-bba9-ec5c950cef6e]",NaN
f0271e34-7791-441b-ad94-a8e2e7f1f909,Single,Results,NCT01644890,cohort 1 of the primary clinical trial had an ...,Contradiction,Contradiction,"[Altering, 8c6314f7-34f5-42a7-ba66-b95d050a6315]",NaN
11073b50-f135-48a1-a8ac-333375a49c14,Single,Eligibility,NCT02186015,children are allowed to participate in the pri...,Contradiction,Contradiction,"[Altering, a805f174-2707-4302-a195-05153ea8523e]",NaN


In [18]:
dataset_f['Intervention'].unique()

array(['Contradiction', 'Numerical_contradiction'], dtype=object)

In [19]:
import pandas as pd
import json


original_dataset['Primary Interventions'] = ''
original_dataset['Primary Eligibility'] = ''
original_dataset['Primary Results'] = ''
original_dataset['Primary Adverse Events'] = ''
original_dataset['Secondary Interventions'] = ''
original_dataset['Secondary Eligibility'] = ''
original_dataset['Secondary Results'] = ''
original_dataset['Secondary Adverse Events'] = ''


dataset_f['Primary Interventions'] = ''
dataset_f['Primary Eligibility'] = ''
dataset_f['Primary Results'] = ''
dataset_f['Primary Adverse Events'] = ''
dataset_f['Secondary Interventions'] = ''
dataset_f['Secondary Eligibility'] = ''
dataset_f['Secondary Results'] = ''
dataset_f['Secondary Adverse Events'] = ''

dataset_c['Primary Interventions'] = ''
dataset_c['Primary Eligibility'] = ''
dataset_c['Primary Results'] = ''
dataset_c['Primary Adverse Events'] = ''
dataset_c['Secondary Interventions'] = ''
dataset_c['Secondary Eligibility'] = ''
dataset_c['Secondary Results'] = ''
dataset_c['Secondary Adverse Events'] = ''



def extract_info_from_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)

    interventions = data.get("Intervention", 'Not Available')
    eligibility_criteria = data.get("Eligibility", 'Not Available')
    results = data.get("Results", 'Not Available')
    adverse_events = data.get('Adverse Events','Not Available')
    return interventions, eligibility_criteria, results, adverse_events


for index, row in original_dataset.iterrows():

    primary_file_path = f"/Users/vikkyfury/Desktop/NLP_group_project/training_data/CT json/{row['Primary_id']}.json"
    secondary_file_path = f"/Users/vikkyfury/Desktop/NLP_group_project/training_data/CT json/{row['Secondary_id']}.json"


    primary_interventions, primary_eligibility, primary_results, primary_adverse_events  = extract_info_from_json(primary_file_path)


    original_dataset.at[index, 'Primary Interventions'] = primary_interventions
    original_dataset.at[index, 'Primary Eligibility'] = primary_eligibility
    original_dataset.at[index, 'Primary Results'] = primary_results
    original_dataset.at[index, 'Primary Adverse Events'] = primary_adverse_events

    if pd.notnull(row['Secondary_id']):
        secondary_interventions, secondary_eligibility, secondary_results, secondary_adverse_events = extract_info_from_json(secondary_file_path)
        original_dataset.at[index, 'Secondary Interventions'] = secondary_interventions
        original_dataset.at[index, 'Secondary Eligibility'] = secondary_eligibility
        original_dataset.at[index, 'Secondary Results'] = secondary_results
        original_dataset.at[index, 'Secondary Adverse Events'] = secondary_adverse_events

for index, row in dataset_f.iterrows():

    primary_file_path = f"/Users/vikkyfury/Desktop/NLP_group_project/training_data/CT json/{row['Primary_id']}.json"
    secondary_file_path = f"/Users/vikkyfury/Desktop/NLP_group_project/training_data/CT json/{row['Secondary_id']}.json"


    primary_interventions, primary_eligibility, primary_results, primary_adverse_events  = extract_info_from_json(primary_file_path)


    dataset_f.at[index, 'Primary Interventions'] = primary_interventions
    dataset_f.at[index, 'Primary Eligibility'] = primary_eligibility
    dataset_f.at[index, 'Primary Results'] = primary_results
    dataset_f.at[index, 'Primary Adverse Events'] = primary_adverse_events

    if pd.notnull(row['Secondary_id']):
        secondary_interventions, secondary_eligibility, secondary_results, secondary_adverse_events = extract_info_from_json(secondary_file_path)
        dataset_f.at[index, 'Secondary Interventions'] = secondary_interventions
        dataset_f.at[index, 'Secondary Eligibility'] = secondary_eligibility
        dataset_f.at[index, 'Secondary Results'] = secondary_results
        dataset_f.at[index, 'Secondary Adverse Events'] = secondary_adverse_events

for index, row in dataset_c.iterrows():

    primary_file_path = f"/Users/vikkyfury/Desktop/NLP_group_project/training_data/CT json/{row['Primary_id']}.json"
    secondary_file_path = f"/Users/vikkyfury/Desktop/NLP_group_project/training_data/CT json/{row['Secondary_id']}.json"


    primary_interventions, primary_eligibility, primary_results, primary_adverse_events  = extract_info_from_json(primary_file_path)


    dataset_c.at[index, 'Primary Interventions'] = primary_interventions
    dataset_c.at[index, 'Primary Eligibility'] = primary_eligibility
    dataset_c.at[index, 'Primary Results'] = primary_results
    dataset_c.at[index, 'Primary Adverse Events'] = primary_adverse_events

    if pd.notnull(row['Secondary_id']):
        secondary_interventions, secondary_eligibility, secondary_results, secondary_adverse_events = extract_info_from_json(secondary_file_path)
        dataset_c.at[index, 'Secondary Interventions'] = secondary_interventions
        dataset_c.at[index, 'Secondary Eligibility'] = secondary_eligibility
        dataset_c.at[index, 'Secondary Results'] = secondary_results
        dataset_c.at[index, 'Secondary Adverse Events'] = secondary_adverse_events





/var/folders/xr/ww1bx5dx03ncctbbx4zptmcm0000gn/T/ipykernel_8887/3493887109.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original_dataset['Primary Interventions'] = ''
/var/folders/xr/ww1bx5dx03ncctbbx4zptmcm0000gn/T/ipykernel_8887/3493887109.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original_dataset['Primary Eligibility'] = ''
/var/folders/xr/ww1bx5dx03ncctbbx4zptmcm0000gn/T/ipykernel_8887/3493887109.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [20]:
def set_evidence(row):
    if row['Section_id'] == 'Intervention':

        if pd.isna(row['Secondary_id']):

            row['primary_evidence'] = row['Primary Interventions']
            row['secondary_evidence'] = None
        else:

            row['primary_evidence'] = row['Primary Interventions']
            row['secondary_evidence'] = row['Secondary Interventions']
    elif row['Section_id'] == 'Eligibility':

        if pd.isna(row['Secondary_id']):

            row['primary_evidence'] = row['Primary Eligibility']
            row['secondary_evidence'] = None
        else:

            row['primary_evidence'] = row['Primary Eligibility']
            row['secondary_evidence'] = row['Secondary Eligibility']
    elif row['Section_id'] == 'Results':
        if pd.isna(row['Secondary_id']):

            row['primary_evidence'] = row['Primary Results']
            row['secondary_evidence'] = None
        else:

            row['primary_evidence'] = row['Primary Results']
            row['secondary_evidence'] = row['Secondary Results']
    elif row['Section_id'] == 'Adverse Events':
        if pd.isna(row['Secondary_id']):

            row['primary_evidence'] = row['Primary Adverse Events']
            row['secondary_evidence'] = None
        else:

            row['primary_evidence'] = row['Primary Adverse Events']
            row['secondary_evidence'] = row['Secondary Adverse Events']

    else:

        pass
    return row

original_dataset =original_dataset.apply(set_evidence, axis=1)
dataset_f = dataset_f.apply(set_evidence, axis=1)
dataset_c = dataset_c.apply(set_evidence, axis=1)

In [21]:
original_dataset.drop(['Primary Interventions','Primary Eligibility','Primary Results',	'Primary Adverse Events','Secondary Interventions',	'Secondary Eligibility','Secondary Results','Secondary Adverse Events'],axis=1,inplace=True)
dataset_f.drop(['Primary Interventions','Primary Eligibility','Primary Results',	'Primary Adverse Events','Secondary Interventions',	'Secondary Eligibility','Secondary Results','Secondary Adverse Events'],axis=1,inplace=True)
dataset_c.drop(['Primary Interventions','Primary Eligibility','Primary Results',	'Primary Adverse Events','Secondary Interventions',	'Secondary Eligibility','Secondary Results','Secondary Adverse Events'],axis=1,inplace=True)

In [22]:
original_dataset.drop(['Intervention','Causal_type'],axis=1,inplace=True)


In [23]:
dataset_f.head(1)

,Type,Section_id,Primary_id,Statement,Label,Intervention,Causal_type,Secondary_id,primary_evidence,secondary_evidence
14e55d50-c012-4f32-afc5-6f5f7a030667,Single,Adverse Events,NCT00328783,the primary trial documents a comprehensive li...,Contradiction,Contradiction,"[Altering, fef5aedb-21c0-4130-b8a3-cb1df5017c7a]",NaN,"[Adverse Events 1:, Total: 0/0]",None


In [24]:
original_dataset.insert(0, 'Index', original_dataset.index)
dataset_f.insert(0, 'Index', dataset_f.index)
dataset_c.insert(0, 'Index', dataset_c.index)

In [25]:
def generate_instruction(row):

    if pd.isnull(row['Secondary_id']):
        premise = f"Primary trial evidence are {row['primary_evidence']}."
    else:
        premise = f"Primary trial evidence are {row['primary_evidence']} and Secondary trial evidence are {row['secondary_evidence']}."


    hypothesis = row['Statement']


    options = "OPTIONS:\nEntailment\nContradiction"


    instruction = f"{premise} Question: Does this imply that {hypothesis}? {options}"
    return instruction

original_dataset['Instruction'] = original_dataset.apply(generate_instruction, axis=1)
dataset_f['Instruction'] = dataset_f.apply(generate_instruction, axis=1)
dataset_c['Instruction'] = dataset_c.apply(generate_instruction, axis=1)

In [26]:
'''custom_stopwords=['a','an','the','are','it','what','which','who','whom','this',
                  'that','these','those','is','was', 'were', 'be', 'been', 'being',
                  'have', 'has', 'had', 'having','if', 'or','as', 'until',
                  'while', 'of', 'at', 'by', 'for', 'with', 'about',
                  'between', 'into', 'through', 'above',
                  'below','to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over',
                   'again','then', 'once', 'here', 'there', 'when',
                  'where', 'why', 'how', 'all','any', 'each',
                  'most', 'other', 'some', 'such', 'only','own', 'so', 'than',
                  'too', 'very', 's', 't', 'can', 'will', 'just', 'should', 'now','it','and','they','their','whereas']'''

"custom_stopwords=['a','an','the','are','it','what','which','who','whom','this',\n                  'that','these','those','is','was', 'were', 'be', 'been', 'being',\n                  'have', 'has', 'had', 'having','if', 'or','as', 'until',\n                  'while', 'of', 'at', 'by', 'for', 'with', 'about',\n                  'between', 'into', 'through', 'above',\n                  'below','to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over',\n                   'again','then', 'once', 'here', 'there', 'when',\n                  'where', 'why', 'how', 'all','any', 'each',\n                  'most', 'other', 'some', 'such', 'only','own', 'so', 'than',\n                  'too', 'very', 's', 't', 'can', 'will', 'just', 'should', 'now','it','and','they','their','whereas']"

In [27]:
'''def remove_custom_stop_words_list(evidence_list):
  try:
    return [' '.join([word for word in item.split() if word not in custom_stopwords]) for item in evidence_list]
  except:
    return None
original_dataset['primary_evidence'] = original_dataset['primary_evidence'].apply(remove_custom_stop_words_list)
original_dataset['secondary_evidence'] = original_dataset['secondary_evidence'].apply(remove_custom_stop_words_list)
dataset_f['primary_evidence'] = dataset_f['primary_evidence'].apply(remove_custom_stop_words_list)
dataset_f['secondary_evidence'] = dataset_f['secondary_evidence'].apply(remove_custom_stop_words_list)
dataset_c['primary_evidence'] = dataset_c['primary_evidence'].apply(remove_custom_stop_words_list)
dataset_c['secondary_evidence'] = dataset_c['secondary_evidence'].apply(remove_custom_stop_words_list)'''


"def remove_custom_stop_words_list(evidence_list):\n  try:\n    return [' '.join([word for word in item.split() if word not in custom_stopwords]) for item in evidence_list]\n  except:\n    return None\noriginal_dataset['primary_evidence'] = original_dataset['primary_evidence'].apply(remove_custom_stop_words_list)\noriginal_dataset['secondary_evidence'] = original_dataset['secondary_evidence'].apply(remove_custom_stop_words_list)\ndataset_f['primary_evidence'] = dataset_f['primary_evidence'].apply(remove_custom_stop_words_list)\ndataset_f['secondary_evidence'] = dataset_f['secondary_evidence'].apply(remove_custom_stop_words_list)\ndataset_c['primary_evidence'] = dataset_c['primary_evidence'].apply(remove_custom_stop_words_list)\ndataset_c['secondary_evidence'] = dataset_c['secondary_evidence'].apply(remove_custom_stop_words_list)"

In [28]:
original_t=original_dataset[['Index','Instruction']]
original_t.shape

(200, 2)

In [29]:
dataset_tf=dataset_f[['Index','Instruction']]
dataset_tf.shape

(678, 2)

In [30]:
dataset_tc=dataset_c[['Index','Instruction']]
dataset_tc.shape

(1264, 2)

In [31]:

def convert_to_dict(row):

    index = row['Index']
    instruction = row['Instruction']


    formatted_entry = f'"{index}": "{instruction}"'

    return formatted_entry

print('for original_t')
print('\n')


converted_entries_original_t = {}

for index, row in original_t.iterrows():

    converted_entries_original_t[index] = convert_to_dict(row)


for key, value in converted_entries_original_t.items():
    print(value)

print('--------------------------------------------------------------------------------')
print('for dataset_tf')
print('\n')

converted_entries_dataset_tf = {}


for index, row in dataset_tf.iterrows():

    converted_entries_dataset_tf[index] = convert_to_dict(row)


for key, value in converted_entries_dataset_tf.items():
    print(value)

print('--------------------------------------------------------------------------------')
print('for dataset_tc')
print('\n')

converted_entries_dataset_tc = {}


for index, row in dataset_tc.iterrows():

    converted_entries_dataset_tc[index] = convert_to_dict(row)


for key, value in converted_entries_dataset_tc.items():
    print(value)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [32]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")


data = converted_entries_dataset_tf


def make_prediction(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=5000)
    outputs = model.generate(**inputs, max_length=20, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


predictions_dataconverted_entries_dataset_tf = {}

for key, text in data.items():
    prediction = make_prediction(text)
    predictions_dataconverted_entries_dataset_tf[key] = {"Prediction": prediction}

print(predictions_dataconverted_entries_dataset_tf)


{'14e55d50-c012-4f32-afc5-6f5f7a030667': {'Prediction': 'no'}, 'fdbb037a-8084-4267-9143-f989d26726ab': {'Prediction': 'yes'}, '64ab5bcd-229a-4aac-8d01-609e514400d7': {'Prediction': 'Yes'}, 'f0271e34-7791-441b-ad94-a8e2e7f1f909': {'Prediction': 'No'}, '11073b50-f135-48a1-a8ac-333375a49c14': {'Prediction': 'Yes'}, '96237d2a-e7f5-4582-b1a9-1dfb233b980d': {'Prediction': 'Yes'}, '78ac8091-0f1b-44d1-a760-71bfdc2d46ff': {'Prediction': 'no'}, '33726f21-bdaa-48a2-a431-c61a84511f7c': {'Prediction': 'Yes'}, 'f364bafe-7930-4aae-8ca5-8494d115fe6a': {'Prediction': 'Yes'}, 'fdc0dbd2-c722-4b99-be3c-602046ef53d2': {'Prediction': 'No'}, '8d01a287-19f8-4403-8b1d-eb40b70f3254': {'Prediction': 'Entailment'}, '7901f050-9868-48c7-b64b-e3f7c7856ed9': {'Prediction': 'No'}, 'dbdb4be1-04c7-4ffe-bcbd-6de8ca84c231': {'Prediction': 'Yes'}, '00c0d4a6-2223-4cc7-ad9f-f5366f5705bb': {'Prediction': 'No'}, '1b5a2128-4e50-4dc7-bba8-16d458bc5256': {'Prediction': 'No'}, '839fd2a0-24d0-417f-9c37-7fb960860547': {'Prediction':

In [33]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")


data = converted_entries_dataset_tc


def make_prediction(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=5000)
    outputs = model.generate(**inputs, max_length=20, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


predictions_dataconverted_entries_dataset_tc = {}

for key, text in data.items():
    prediction = make_prediction(text)
    predictions_dataconverted_entries_dataset_tc[key] = {"Prediction": prediction}

print(predictions_dataconverted_entries_dataset_tc)


{'ea57b54d-1912-4d9f-bcf7-7f423aed195b': {'Prediction': 'Yes'}, '6c337426-cff6-4f2c-acab-e7bf15f09cc2': {'Prediction': 'yes'}, '141798cc-9851-4bdf-b8a9-ce688db34e72': {'Prediction': 'yes'}, '47f4c385-4dd8-4b6e-a365-bb2fac258ab9': {'Prediction': 'yes'}, 'd31f179f-1253-44c8-8fc6-167f7b3d41db': {'Prediction': 'Yes'}, '76236597-7a5e-4042-b1c4-ddd5a4aa57e7': {'Prediction': 'No'}, '572c52ba-9518-4b72-9a32-a08604808764': {'Prediction': 'no'}, 'ec5ca1ce-8399-4213-9f44-53e55b7352db': {'Prediction': 'Contradiction'}, '95f5af8f-1590-4254-bd9f-885b6720b7b5': {'Prediction': 'Yes'}, '41a49fe6-55b7-4f89-bc8c-b0107350a79a': {'Prediction': 'Entailment'}, '764bd30d-89e3-45d5-b385-95b63322a043': {'Prediction': 'no'}, '7bd16537-caf9-4749-bb7d-bdc5207a5ac2': {'Prediction': 'no'}, '3de38124-1362-45d3-8ee3-ea39c1331315': {'Prediction': 'no'}, '699fe4af-e23b-4d8d-b9e8-aa8a722d5a22': {'Prediction': 'Yes'}, '3b28a747-dad6-4b2f-bdf0-c7678b3b0506': {'Prediction': 'Yes'}, 'ffb356e8-2d61-4f1d-9ab9-978c7a723172': {'

In [34]:
faith_fulness = dataset_f[['Label']]
faith_fulness

,Label
14e55d50-c012-4f32-afc5-6f5f7a030667,Contradiction
fdbb037a-8084-4267-9143-f989d26726ab,Contradiction
64ab5bcd-229a-4aac-8d01-609e514400d7,Contradiction
f0271e34-7791-441b-ad94-a8e2e7f1f909,Contradiction
11073b50-f135-48a1-a8ac-333375a49c14,Contradiction
...,...
8bbedcf5-cbaa-45f4-b67d-73e06a0d8e1f,Contradiction
e055e769-39a4-44b3-b7a1-79ae72545c75,Contradiction
c3b6337c-855a-4ba0-99bd-1e6ab8f1b0e2,Contradiction
60e609fe-c70e-4916-93b8-b5446ac8a937,Contradiction


In [35]:
dataset_f['Label'].unique()

array(['Contradiction'], dtype=object)

In [36]:
consis_tency = dataset_c[['Label']]
consis_tency

,Label
ea57b54d-1912-4d9f-bcf7-7f423aed195b,Entailment
6c337426-cff6-4f2c-acab-e7bf15f09cc2,Contradiction
141798cc-9851-4bdf-b8a9-ce688db34e72,Entailment
47f4c385-4dd8-4b6e-a365-bb2fac258ab9,Contradiction
d31f179f-1253-44c8-8fc6-167f7b3d41db,Entailment
...,...
19e110af-a2cd-4f49-a97b-57d5ef932bb2,Contradiction
d284b8b2-b2bb-4f1f-a622-9aab2b91e122,Entailment
65cd1454-3297-4ede-bbfe-7bdcf089e40b,Entailment
413a1496-e87b-46c0-abd6-5cef0f23352f,Entailment


In [37]:
contrast_list = [value['Prediction'] for value in predictions_dataconverted_entries_dataset_tf.values()]
contrast_list


['no',
 'yes',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'no',
 'Yes',
 'Yes',
 'No',
 'Entailment',
 'No',
 'Yes',
 'No',
 'No',
 'Entailment',
 'Yes',
 'Yes',
 'Yes',
 'no',
 'no',
 'No',
 'Yes',
 'Contradiction',
 'no',
 'Yes',
 'Yes',
 'Yes',
 'no',
 'Contradiction',
 'yes',
 'No',
 'No',
 'Yes',
 'Entailment',
 'No',
 'Entailment',
 'Yes',
 'Yes',
 'Yes',
 'no',
 'No',
 'yes',
 'yes',
 'Entailment',
 'Yes',
 'No',
 'Yes',
 'no',
 'Yes',
 'yes',
 'Yes',
 'no',
 'Contradiction',
 'Yes',
 'no',
 'Yes',
 'Entailment',
 'no',
 'No',
 'Yes',
 'No',
 'Contradiction',
 'Yes',
 'yes',
 'no',
 'No',
 'yes',
 'Yes',
 'Contradiction',
 'No',
 'no',
 'no',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'No',
 'no',
 'No',
 'Yes',
 'Yes',
 'No',
 'Contradiction',
 'Entailment',
 'yes',
 'yes',
 'No',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'no',
 'Yes',
 'Contradiction',
 'No',
 'No',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'yes',
 'N

In [38]:
contrast_list = [0 if value.lower() in ['no', 'contradiction'] else 1 for value in contrast_list]

print(contrast_list)

[0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 

In [39]:
consis_list = [value['Prediction'] for value in predictions_dataconverted_entries_dataset_tc.values()]



In [40]:
consis_list = [0 if value.lower() in ['no', 'contradiction'] else 1 for value in consis_list]

print(consis_list)

[1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 

In [41]:
o_f_labels = faith_fulness['Label'].tolist()


In [42]:
o_f_labels = [1 for value in o_f_labels]
o_f_labels

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [43]:
o_c_labels = consis_tency['Label'].tolist()
o_c_labels

['Entailment',
 'Contradiction',
 'Entailment',
 'Contradiction',
 'Entailment',
 'Contradiction',
 'Contradiction',
 'Contradiction',
 'Entailment',
 'Contradiction',
 'Entailment',
 'Entailment',
 'Contradiction',
 'Contradiction',
 'Entailment',
 'Contradiction',
 'Contradiction',
 'Contradiction',
 'Entailment',
 'Contradiction',
 'Entailment',
 'Entailment',
 'Contradiction',
 'Contradiction',
 'Entailment',
 'Entailment',
 'Contradiction',
 'Contradiction',
 'Entailment',
 'Contradiction',
 'Entailment',
 'Entailment',
 'Contradiction',
 'Contradiction',
 'Contradiction',
 'Contradiction',
 'Entailment',
 'Entailment',
 'Contradiction',
 'Entailment',
 'Entailment',
 'Contradiction',
 'Entailment',
 'Contradiction',
 'Entailment',
 'Entailment',
 'Contradiction',
 'Entailment',
 'Contradiction',
 'Contradiction',
 'Contradiction',
 'Contradiction',
 'Contradiction',
 'Contradiction',
 'Contradiction',
 'Entailment',
 'Contradiction',
 'Contradiction',
 'Contradiction',
 'Entailme

In [44]:
o_c_labels = [1 if value=='Entailment' else 0 for value in o_c_labels]
o_c_labels

[1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,


In [45]:
def Faithfulness(original_preds, contrast_preds):
    if len(original_preds) != len(contrast_preds):
        raise ValueError("The lists of predictions must have the same length.")
    faithfulness = sum(abs(o - c) for o, c in zip(original_preds, contrast_preds)) / len(original_preds)
    return faithfulness
def Consistency(original_preds, contrast_preds):
    if len(original_preds) != len(contrast_preds):
        raise ValueError("The lists of predictions must have the same length.")
    consistency = sum(1-abs(o - c) for o, c in zip(original_preds, contrast_preds)) / len(original_preds)
    return consistency

In [46]:
faithfulness_score = Faithfulness(o_f_labels,contrast_list)
consistency_score = Consistency(o_c_labels,consis_list)
print('Faithfulness score is :',faithfulness_score)
print('Consistency score is:',consistency_score)

Faithfulness score is : 0.41150442477876104
Consistency score is: 0.5181962025316456
